In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

pd.options.display.float_format = '{:.2f}'.format  #pandas 取消科學記號，只顯示小數點第二位

In [2]:
bureau=pd.read_csv("D:/project/homecredit/bureau.csv") 
train = pd.read_csv("D:/project/homecredit/application_train.csv")

In [3]:
bureau.shape

(1716428, 17)

In [4]:
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_columns',None)
bureau=bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU'])

In [5]:
bureau.isna().sum()/bureau.shape[0]

SK_ID_CURR               0.00
SK_ID_BUREAU             0.00
CREDIT_ACTIVE            0.00
CREDIT_CURRENCY          0.00
DAYS_CREDIT              0.00
CREDIT_DAY_OVERDUE       0.00
DAYS_CREDIT_ENDDATE      0.06
DAYS_ENDDATE_FACT        0.37
AMT_CREDIT_MAX_OVERDUE   0.66
CNT_CREDIT_PROLONG       0.00
AMT_CREDIT_SUM           0.00
AMT_CREDIT_SUM_DEBT      0.15
AMT_CREDIT_SUM_LIMIT     0.34
AMT_CREDIT_SUM_OVERDUE   0.00
CREDIT_TYPE              0.00
DAYS_CREDIT_UPDATE       0.00
AMT_ANNUITY              0.71
dtype: float64

In [6]:
bureau_fill = bureau.copy()

In [7]:
app_train_target = train[['SK_ID_CURR','TARGET','EXT_SOURCE_3']]
app_train_target.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3
0,100002,1,0.14
1,100003,0,NaN
2,100004,0,0.73
3,100006,0,NaN
4,100007,0,NaN


In [8]:
bureau_train_target=pd.merge(bureau, app_train_target,how='left')
bureau_train_target.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,EXT_SOURCE_3
0,100001,5896630,Closed,currency 1,-857,0,-492.00,-553.00,NaN,0,112500.00,0.00,0.00,0.00,Consumer credit,-155,0.00,NaN,NaN
1,100001,5896631,Closed,currency 1,-909,0,-179.00,-877.00,NaN,0,279720.00,0.00,0.00,0.00,Consumer credit,-155,0.00,NaN,NaN
2,100001,5896632,Closed,currency 1,-879,0,-514.00,-544.00,NaN,0,91620.00,0.00,0.00,0.00,Consumer credit,-155,0.00,NaN,NaN
3,100001,5896633,Closed,currency 1,-1572,0,-1329.00,-1328.00,NaN,0,85500.00,0.00,0.00,0.00,Consumer credit,-155,0.00,NaN,NaN
4,100001,5896634,Active,currency 1,-559,0,902.00,NaN,NaN,0,337680.00,113166.00,0.00,0.00,Consumer credit,-6,4630.50,NaN,NaN


In [9]:
bur1 = bureau.sort_values(by=['SK_ID_CURR','SK_ID_BUREAU']).copy()


#a1：新增5個欄位- CREDIT_ACTIVE_(Active、Bad debt、Closed、Sold、all)，將同一個'SK_ID_CURR'彙整，列出'SK_ID_BUREAU'的CREDIT_ACTIVE發生的次數。

#列出同一個SK_ID_BUREAU，發生STATUS不同情況的次數
bur1_1=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_ACTIVE',aggfunc=['count'], fill_value=0)
bur1_1.rename(columns={'Active':'CREDIT_ACTIVE_Active_count',
                       'Bad debt':'CREDIT_ACTIVE_Bad_debt_count',
                       'Closed':'CREDIT_ACTIVE_Closed_count',
                       'Sold':'CREDIT_ACTIVE_Sold_count'},inplace=True)  


#a3：新增5個欄位- CREDIT_CURRENCY_currency(1、2、3、4、all))，將同一個'SK_ID_CURR'彙整，列出SK_ID_BUREAU'的CREDIT_CURRENCY發生的次數。
bur1_2=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_CURRENCY',aggfunc=['count'], fill_value=0)
bur1_2.rename(columns={ 'currency 1':'CREDIT_CURRENCY_1_count',
                        'currency 2':'CREDIT_CURRENCY_2_count',
                        'currency 3':'CREDIT_CURRENCY_3_count',
                        'currency 4':'CREDIT_CURRENCY_4_count'},inplace=True) 


#a5：新增16個欄位- CREDIT_TYPE (Another type of loan、Car loan...、all))，將同一個'SK_ID_CURR'彙整，列出CREDIT_TYPE發生的次數。
bur1_3=bur1.pivot_table(values=['SK_ID_BUREAU'],index='SK_ID_CURR',columns='CREDIT_TYPE',aggfunc=['count'], fill_value=0)
bur1_3.rename(columns={ 'Another type of loan':'CREDIT_TYPE_Another_type_of_loan_count',
                        'Car loan':'CREDIT_TYPE_Car_loan_count',
                        'Cash loan (non-earmarked)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                        'Consumer credit':'CREDIT_TYPE_Consumer_credit_count',
                        'Credit card':'CREDIT_TYPE_Credit_card_count',
                        'Interbank credit':'CREDIT_TYPE_Interbank_credit_count',
                        'Loan for business development':'CREDIT_TYPE_Loan_for_business_development_count',
                        'Loan for purchase of shares (margin lending)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                        'Loan for the purchase of equipment':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                        'Loan for working capital replenishment':'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                        'Microloan':'CREDIT_TYPE_Microloan_count',
                        'Mobile operator loan':'CREDIT_TYPE_Mobile_operator_loan_count',
                        'Mortgage':'CREDIT_TYPE_Mortgage_count',
                        'Real estate loan':'CREDIT_TYPE_Real_estate_loan_count',
                        'Unknown type of loan':'CREDIT_TYPE_Unknown_type_of_loan_count'},inplace=True)



                                            


In [10]:
bur2 = bureau.sort_values(by=['SK_ID_CURR' , 'SK_ID_BUREAU']).copy()

bur2['DEBT/CREDIT'] = bur2['AMT_CREDIT_SUM_DEBT']/bur2['AMT_CREDIT_SUM']
bur2['SUM_OVERDUE/CREDIT'] = bur2['AMT_CREDIT_SUM_OVERDUE']/bur2['AMT_CREDIT_SUM']
bur2['MAX_OVERDUE/CREDIT'] = bur2['AMT_CREDIT_MAX_OVERDUE']/bur2['AMT_CREDIT_SUM']
bur2['ANNUITY/CREDIT'] = bur2['AMT_ANNUITY']/bur2['AMT_CREDIT_SUM']



bur2['DAYS_CREDIT_recent']=bur2['DAYS_CREDIT']

bur2['DAYS_CREDIT_farest']=bur2['DAYS_CREDIT']

bur2['DAYS_CREDIT_mean']=bur2['DAYS_CREDIT']

bur2['CREDIT_DAY_OVERDUE_count']=(bur2['CREDIT_DAY_OVERDUE']>0).replace(True,1)
bur2['CREDIT_DAY_OVERDUE_min']=bur2['CREDIT_DAY_OVERDUE']
bur2['CREDIT_DAY_OVERDUE_max']=bur2['CREDIT_DAY_OVERDUE']
bur2['CREDIT_DAY_OVERDUE_mean']=bur2['CREDIT_DAY_OVERDUE']

bur2['DAYS_CREDIT_ENDDATE_max']=bur2['DAYS_CREDIT_ENDDATE']
bur2['DAYS_CREDIT_ENDDATE_min']=bur2['DAYS_CREDIT_ENDDATE']
bur2['DAYS_CREDIT_ENDDATE_median']=bur2['DAYS_CREDIT_ENDDATE']

bur2['DAYS_ENDDATE_FACT_max']=bur2['DAYS_ENDDATE_FACT']
bur2['DAYS_ENDDATE_FACT_min']=bur2['DAYS_ENDDATE_FACT']
bur2['DAYS_ENDDATE_FACT_mean']=bur2['DAYS_ENDDATE_FACT']

bur2['AMT_CREDIT_MAX_OVERDUE_count']=(bur2['AMT_CREDIT_MAX_OVERDUE']>0).replace(True,1)
bur2['AMT_CREDIT_MAX_OVERDUE_min']=bur2['AMT_CREDIT_MAX_OVERDUE']
bur2['AMT_CREDIT_MAX_OVERDUE_max']=bur2['AMT_CREDIT_MAX_OVERDUE']
bur2['AMT_CREDIT_MAX_OVERDUE_mean']=bur2['AMT_CREDIT_MAX_OVERDUE']

bur2['CNT_CREDIT_PROLONG_count']=(bur2['CNT_CREDIT_PROLONG']>0).replace(True,1)
bur2['CNT_CREDIT_PROLONG_min']=bur2['CNT_CREDIT_PROLONG']
bur2['CNT_CREDIT_PROLONG_max']=bur2['CNT_CREDIT_PROLONG']
bur2['CNT_CREDIT_PROLONG_mean']=bur2['CNT_CREDIT_PROLONG']

bur2['AMT_CREDIT_SUM_DEBT_min']=bur2['AMT_CREDIT_SUM_DEBT']
bur2['AMT_CREDIT_SUM_DEBT_max']=bur2['AMT_CREDIT_SUM_DEBT']
bur2['AMT_CREDIT_SUM_DEBT_mean']=bur2['AMT_CREDIT_SUM_DEBT']

bur2['AMT_CREDIT_SUM_LIMIT_max']=bur2['AMT_CREDIT_SUM_LIMIT']
bur2['AMT_CREDIT_SUM_LIMIT_mean']=bur2['AMT_CREDIT_SUM_LIMIT']

bur2['AMT_CREDIT_SUM_OVERDUE_count']=(bur2['AMT_CREDIT_SUM_OVERDUE']>0).replace(True,1)
bur2['AMT_CREDIT_SUM_OVERDUE_min']=bur2['AMT_CREDIT_SUM_OVERDUE']
bur2['AMT_CREDIT_SUM_OVERDUE_max']=bur2['AMT_CREDIT_SUM_OVERDUE']
bur2['AMT_CREDIT_SUM_OVERDUE_mean']=bur2['AMT_CREDIT_SUM_OVERDUE']

bur2['DAYS_CREDIT_UPDATE_max']=bur2['DAYS_CREDIT_UPDATE']
bur2['DAYS_CREDIT_UPDATE_median']=bur2['DAYS_CREDIT_UPDATE']

bur2['DEBT/CREDIT_max']=bur2['DEBT/CREDIT']
bur2['DEBT/CREDIT_mean']=bur2['DEBT/CREDIT']

bur2['SUM_OVERDUE/CREDIT_max']=bur2['SUM_OVERDUE/CREDIT']
bur2['SUM_OVERDUE/CREDIT_mean']=bur2['SUM_OVERDUE/CREDIT']

bur2['MAX_OVERDUE/CREDIT_max']=bur2['MAX_OVERDUE/CREDIT']
bur2['MAX_OVERDUE/CREDIT_mean']=bur2['MAX_OVERDUE/CREDIT']


bur2['AMT_CREDIT_SUM_max'] = bur2['AMT_CREDIT_SUM']
bur2['AMT_CREDIT_SUM_min'] = bur2['AMT_CREDIT_SUM']

bur2['AMT_ANNUITY_max']=bur2['AMT_ANNUITY']
bur2['AMT_ANNUITY_min']=bur2['AMT_ANNUITY']

bur2['ANNUITY/CREDIT_max']=bur2['ANNUITY/CREDIT']
bur2['ANNUITY/CREDIT_min']=bur2['ANNUITY/CREDIT']


# 創造新的欄位 
num_aggregations = {
        'DAYS_CREDIT_recent':'max',
        'DAYS_CREDIT_farest':'min',
        'DAYS_CREDIT_mean':'mean',
        'CREDIT_DAY_OVERDUE_count':'sum',
        'CREDIT_DAY_OVERDUE_min':'min',
        'CREDIT_DAY_OVERDUE_max':'max',
        'CREDIT_DAY_OVERDUE_mean':'mean',
        'DAYS_CREDIT_ENDDATE_max':'max',
        'DAYS_CREDIT_ENDDATE_min':'min',
        'DAYS_CREDIT_ENDDATE_median':'median',
        'DAYS_ENDDATE_FACT_max':'max',
        'DAYS_ENDDATE_FACT_min':'min',
        'DAYS_ENDDATE_FACT_mean':'mean',
        'AMT_CREDIT_MAX_OVERDUE_count':'sum',        
        'AMT_CREDIT_MAX_OVERDUE_min':'min',
        'AMT_CREDIT_MAX_OVERDUE_max':'max',
        'AMT_CREDIT_MAX_OVERDUE_mean':'mean',
        'CNT_CREDIT_PROLONG_count':'sum',
        'CNT_CREDIT_PROLONG_min':'min',
        'CNT_CREDIT_PROLONG_max':'max',
        'CNT_CREDIT_PROLONG_mean':'mean',
        'AMT_CREDIT_SUM_DEBT_min':'min',
        'AMT_CREDIT_SUM_DEBT_max':'max',
        'AMT_CREDIT_SUM_DEBT_mean':'mean',
        'AMT_CREDIT_SUM_LIMIT_max':'max',
        'AMT_CREDIT_SUM_LIMIT_mean':'mean',
        'AMT_CREDIT_SUM_OVERDUE_count':'sum',
        'AMT_CREDIT_SUM_OVERDUE_min':'min',
        'AMT_CREDIT_SUM_OVERDUE_max':'max',
        'AMT_CREDIT_SUM_OVERDUE_mean':'mean',
        'DAYS_CREDIT_UPDATE_max':'max',
        'DAYS_CREDIT_UPDATE_median':'median',
        'DEBT/CREDIT_max':'max',
        'DEBT/CREDIT_mean':'mean',
        'SUM_OVERDUE/CREDIT_max':'max',
        'SUM_OVERDUE/CREDIT_mean':'mean',
        'MAX_OVERDUE/CREDIT_max':'max',
        'MAX_OVERDUE/CREDIT_mean':'mean',
        'AMT_CREDIT_SUM_max':'max',
        'AMT_CREDIT_SUM_min':'min',
        'AMT_ANNUITY_max':'max',
        'AMT_ANNUITY_min':'min',
        'ANNUITY/CREDIT_max':'max',
        'ANNUITY/CREDIT_min':'min',

    }

bur2_1 = bur2.groupby(['SK_ID_CURR']).agg({**num_aggregations})

# 重新排列
#bur2_1 .reset_index(level=('SK_ID_CURR'),inplace=True)






bur2_1.head(20)
bur2_1.shape

(305811, 44)

In [11]:
#合併所有新增特徵欄位
bur_all=[bur1_1,bur1_2,bur1_3,bur2_1]
bur_all=pd.concat(bur_all , axis=1)

# 重新排列
bur_all.reset_index(level=('SK_ID_CURR'),inplace=True)
bur_all.rename(columns={ '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Active_count)':'CREDIT_ACTIVE_Active_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Bad_debt_count)':'CREDIT_ACTIVE_Bad_debt_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Closed_count)':'CREDIT_ACTIVE_Closed_count',
                         '(count, SK_ID_BUREAU, CREDIT_ACTIVE_Sold_count)':'CREDIT_ACTIVE_Sold_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_1_count)':'CREDIT_CURRENCY_1_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_2_count)':'CREDIT_CURRENCY_2_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_3_count)':'CREDIT_CURRENCY_3_count',
                         '(count, SK_ID_BUREAU, CREDIT_CURRENCY_4_count)':'CREDIT_CURRENCY_4_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Another_type_of_loan_count)':'CREDIT_TYPE_Another_type_of_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Car_loan_count)':'CREDIT_TYPE_Car_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Cash_loan_(non-earmarked)_count)':'CREDIT_TYPE_Cash_loan_(non-earmarked)_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Consumer_credit_count)':'CREDIT_TYPE_Consumer_credit_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Credit_card_count)':' CREDIT_TYPE_Credit_card_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Interbank_credit_count)':'CREDIT_TYPE_Interbank_credit_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_business_development_count)':'CREDIT_TYPE_Loan_for_business_development_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count)':'CREDIT_TYPE_Loan for_purchase_of_shares_(margin lending)_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count)':'CREDIT_TYPE_Loan_for_the_purchase_of_equipment_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Loan_for_working_capital_replenishment_count)':'CREDIT_TYPE_Loan_for_working_capital_replenishment_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Microloan_count)':' CREDIT_TYPE_Microloan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Mobile_operator_loan_count)':'CREDIT_TYPE_Mobile_operator_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Mortgage_count)':'CREDIT_TYPE_Mortgage_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Real_estate_loan_count)':' CREDIT_TYPE_Real_estate_loan_count',
                         '(count, SK_ID_BUREAU, CREDIT_TYPE_Unknown_type_of_loan_count)':'CREDIT_TYPE_Unknown_type_of_loan_count',

                         
                        },inplace=True)
bur_all.shape

(305811, 68)

In [12]:
bur_all.columns

Index([                                                                               'SK_ID_CURR',
                                           ('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Active_count'),
                                         ('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Bad_debt_count'),
                                           ('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Closed_count'),
                                             ('count', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Sold_count'),
                                              ('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_1_count'),
                                              ('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_2_count'),
                                              ('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_3_count'),
                                              ('count', 'SK_ID_BUREAU', 'CREDIT_CURRENCY_4_count'),
                               ('count', 'SK_ID_BUREAU', 'CREDIT_TYPE_Another_type_of_loan_count'),


In [13]:
bur_all.to_csv('D:/project/homecredit/final/bureau_ETL_v2.csv')